In [ ]:
import matplotlib.pyplot as plt
from ModelTools.Metric import Metric
from ModelTools.RegBuilder import RegBuilder
from ModelTools.QRegBuilder import QRegBuilder
from ModelTools._src import tabulate
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

from plotnine.data import mtcars
import plotnine as gg

In [ ]:
mtcars
p1=gg.qplot('mpg','wt',mtcars)
p1

In [ ]:
mtcars

In [ ]:
m1 = RegBuilder('fml_HUBER',formula='~mpg').fit(mtcars,mtcars.wt)
lb,ub = m1.predict_interval()
p1+gg.geom_line(gg.aes(x=mtcars.mpg.to_numpy(),y=m1.predict()))+gg.geom_ribbon(gg.aes(ymax=ub,ymin=lb),alpha=0.3)

In [ ]:
m2=QRegBuilder('fml_GBR',formula='~mpg').fit(mtcars[['mpg']],mtcars.wt,alpha=0.9)
p1+gg.geom_line(gg.aes(x=mtcars.mpg.to_numpy(),y=m2.predict()))

In [ ]:
m2.cv.best_estimator_

In [ ]:
def formula_transform(data:pd.DataFrame,formula:str):
    from formulaic import model_matrix
    return model_matrix(formula,data)  

formula_transform(pd.DataFrame(columns=['x','y'],data=np.zeros(2).reshape(1,-1)),formula='~x')

In [ ]:
np.ones_like(['x','y']).reshape(1,-1)

In [ ]:


x_new = np.linspace(-3,3,100)
x_new = np.linspace(min(x),max(x),100)
y_new = rng.normal(1+np.sin(x_new),scale=0.1)

mean = m1.predict(x_new.reshape(-1,1))
lw,up=m1.predict_interval(x_new.reshape(-1,1),alpha=0.01)

fig,ax = plt.subplots(1,2)

ax[0].plot(x_new,mean,c='red')
ax[0].scatter(x_new,y_new,s=1)
ax[0].fill_between(x_new,lw,up,alpha=0.3)

qr_up = m2.fit(x.reshape(-1,1),y,alpha=0.99).predict(x_new.reshape(-1,1))
qr_lw = m2.fit(x.reshape(-1,1),y,alpha=0.01).predict(x_new.reshape(-1,1))
ax[1].fill_between(x_new,qr_up,qr_lw,alpha=0.3)
ax[1].scatter(x_new,y_new,s=1)

for i in [0,1]:
    ax[i].vlines(x=min(x),ymin=min(lw),ymax=max(up),colors='green')
    ax[i].vlines(x=max(x),ymin=min(lw),ymax=max(up),colors='green')

In [ ]:
qr_up-qr_lw